In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from utils.train_stft import train_cnn_stft
import matplotlib.pyplot as plt
import numpy as np
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.optuna import OptunaSearch
from ray.air import RunConfig

CONFIG = {
    "dropout": tune.choice([0.1,0.2,0.3,0.4]),
    "f_start": tune.uniform(0, 1000),
    "batch_size": tune.choice([16,32,64]),
    "learning_rate": tune.loguniform(1e-5, 1e-2),
    "input_channels": tune.choice([32,64,128]),
}

reporter = CLIReporter(
        parameter_columns=["input_channels", "dropout", "learning_rate", "batch_size", "f_start"],
        metric_columns=["loss", "accuracy", "training_iteration"])
algo = OptunaSearch()

tuner = tune.Tuner(
        tune.with_resources(
                tune.with_parameters(train_cnn_stft),
                resources={"cpu": 16, "gpu": 2}
        ),
        tune_config=tune.TuneConfig(
                metric='accuracy',
                mode="max",
                search_alg=algo,
                num_samples=5,
        ),
        run_config=RunConfig(
                stop={"training_iteration": 60},
                verbose=2,
        ),
        param_space=CONFIG,
)
results = tuner.fit()

# model = models.resnet18(False)
# model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# model.fc = torch.nn.Linear(512, 3)

best_result = results.get_best_result("accuracy", "max")
print("Best config is:", results.get_best_result().config)

/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.8.dylib
  Referenced from: <BE0CCD9A-269A-30E2-A23C-DA45E89EBB1F> /Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/nattapolchanpaisit/miniforge3/lib/python3.10/lib-dynload/../../libjpeg.8.dylib' (no such file), '/Users/nattapolchanpaisit/miniforge3/bin/../lib/libjpeg.8.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS@rpath/libjpeg.8.dylib' (no such file), '/Users/nattapolchanpaisit/miniforge3/li

/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/optuna/distributions.py:799: FutureWarning: UniformDistribution(high=1000.0, low=0.0) is deprecated and internally converted to FloatDistribution(high=1000.0, log=False, low=0.0, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)
/Users/nattapolchanpaisit/miniforge3/lib/python3.10/site-packages/optuna/distributions.py:799: FutureWarning: LogUniformDistribution(high=0.01, low=1e-05) is deprecated and internally converted to FloatDistribution(high=0.01, log=True, low=1e-05, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)


(autoscaler +11s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +11s) Error: No available node types can fulfill resource request {'GPU': 2.0, 'CPU': 16.0}. Add suitable node types to this cluster to resolve this issue.


(raylet) [2023-04-21 20:39:09,937 E 51928 1862874] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-04-21_20-38-49_645051_51895 is over 95% full, available space: 11165229056; capacity: 245107195904. Object creation will fail if spilling is required.
2023-04-21 20:39:17,862	WARNING tune.py:146 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
(raylet) [2023-04-21 20:39:20,037 E 51928 1862874] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-04-21_20-38-49_645051_51895 is over 95% full, available space: 11165167616; capacity: 245107195904. Object creation will fail if spilling is required.
2023-04-21 20:39:20,883	ERROR tune.py:794 -- Trials did not complete: [train_cnn_stft_6d74cb8f]
2023-04-21 20:39:20,883	INFO tune.py:798 -- Total run time: 20.15 seconds (20.13 seconds for the tuning loop).
2023-04-21 20:39:20,884	WARNING tune

Best config is: None


(raylet) [2023-04-21 20:39:30,135 E 51928 1862874] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-04-21_20-38-49_645051_51895 is over 95% full, available space: 11165184000; capacity: 245107195904. Object creation will fail if spilling is required.
(raylet) [2023-04-21 20:39:40,141 E 51928 1862874] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-04-21_20-38-49_645051_51895 is over 95% full, available space: 11165040640; capacity: 245107195904. Object creation will fail if spilling is required.
(raylet) [2023-04-21 20:39:50,239 E 51928 1862874] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-04-21_20-38-49_645051_51895 is over 95% full, available space: 11165036544; capacity: 245107195904. Object creation will fail if spilling is required.
(raylet) [2023-04-21 20:40:00,240 E 51928 1862874] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2023-04-21_20-38-49_645051_51895 is over 95% full, available space: 11165052928; capacity: 245107195904. Object 

In [ ]:
df = results.get_dataframe()
columns_to_drop = ['time_this_iter_s', 'should_checkpoint', 'done', 'timesteps_total',
       'episodes_total', 'training_iteration', 'trial_id', 'experiment_id',
       'date', 'timestamp', 'time_total_s', 'pid', 'hostname', 'node_ip',
       'time_since_restore', 'timesteps_since_restore',
       'iterations_since_restore', 'warmup_time', 'extrastole_val_tpr_numer', 
       'extrastole_val_tpr_denom', 'extrastole_val_fpr_numer', 'extrastole_val_fpr_denom']
for column in columns_to_drop:
    df = df.drop(columns=column)
path = '/'.join(df['logdir'][0].split('/')[:4])
df.to_csv(f'{path}/results.csv')

In [ ]:
df

NameError: name 'df' is not defined